In this notebook we begin learning and exploring how to continuously listen for signals, that we might then pass through our trained neural net in order to recognize specific noises and act on them.

# Learning to use sounddevice: minimal working example

A couple instructive examples that use [pyaudio](https://people.csail.mit.edu/hubert/pyaudio/) can be found here:
* https://github.com/swharden/Python-GUI-examples/blob/master/2016-07-37_qt_audio_monitor/SWHear.py
* https://github.com/chaosparrot/parrot.py/blob/master/lib/listen.py

The latter is a more complex example, but is part of a project similar to this one, and so may be particularly insightful. We will postpone trying to parse it until we need to, however. In particular, first let's understand the basics, and construct a minimal working example.

An alternative library, which seems as powerful but much better documented, is [sounddevice](https://python-sounddevice.readthedocs.io/en/0.3.14/). This also has the benefit of outputting numpy arrays by default, which will save us some processing. We'll try to use sounddevice, but remember pyaudio as a fallback option.

So let's construct a minimal working example. Here we are adapting and stripping down code from https://python-sounddevice.readthedocs.io/en/0.3.14/examples.html, a command-line script which shows a text-mode spectrogram using live microphone data. This listens for five seconds, printing out the maximum amplitude of each 50ms interval.

In [5]:
# Let's see what devices I have
print(sd.query_devices())
sd.query_devices(2, 'input')['default_samplerate']

  0 Built-in Microphone, Core Audio (2 in, 0 out)
< 1 Built-in Output, Core Audio (0 in, 2 out)
> 2 SpeechMatic USB MultiAdapter, Core Audio (1 in, 2 out)


44100.0

In [14]:
import sounddevice as sd
from IPython.display import clear_output
import time

class args:
    block_duration = 50 # ms
    device = 2

samplerate = sd.query_devices(args.device, 'input')['default_samplerate']

def callback(indata, frames, time, status):
    if status:
        print('STATUS: ', str(status))
    if any(indata):
        # dynamically print the max and min values
        clear_output(wait=True) # this sometimes takes too long, causing input overflows
        print(indata.max())
        print(indata.min())
    else:
        print('no input')

start = time.time()
with sd.InputStream(device=args.device, channels=1, callback=callback,
                    blocksize=int(samplerate * args.block_duration / 1000),
                    samplerate=samplerate):
    while True:
        # listen for five seconds
        if time.time() - start > 5:
            break

0.0007324219
-0.00091552734
